In [1]:
import pdb
from math import log10, floor

import numpy as np
import scipy.optimize as op
import scipy.special as spec
from scipy import stats
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
import seaborn as sns
import corner

import astropy
from astropy import stats as astrostats

import emcee
import radvel

%matplotlib inline

In [2]:
sys_props   = pd.read_csv('legacy_tables/system_props.csv')
stellar     = pd.read_csv('legacy_tables/stellar_parameters.csv')
masses_only = pd.read_csv('legacy_tables/planet_list.csv')
trends      = pd.read_csv('legacy_tables/trend_list.csv')
data        = pd.read_csv('../rvdata/legacy_all_data.csv')

# Make table for planet parameters.

In [3]:
masses_real    = masses_only.query('status == "K" or status == "C" or status == "S"').reset_index(drop=True)
masses_good    = masses_real.query('mass != "nan"').reset_index(drop=True)#[masses_only.hostname != 'hip63510']
masses_binary  = masses_good.query('status == "S" or status == "SS"').reset_index(drop=True)
masses_planet  = masses_good.query('status != "S" and status != "SS"').reset_index(drop=True)
masses_old     = masses_planet.query('status == "K"').reset_index(drop=True)
masses_new     = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)
false_positive = masses_only.query('status == "N" or status == "A"  or status == "R"')#[['hostname', 'k', 'per', 'status']]

In [5]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_planet['hostname']
masses_table['Msini (MJup)'] = masses_planet['mass_med']
masses_table['a (AU)'] = masses_planet['axis_med']
masses_table['P (d)'] = masses_planet['per_med']
masses_table['e'] = masses_planet['e_med']

names = np.unique(np.array(masses_table.Name))

nplanets = np.array(masses_table.groupby('Name').count()['e'])
db_nplanets = pd.DataFrame({'Name':names, 'nplanets':nplanets})
masses_table = pd.merge(masses_table, db_nplanets, on='Name').reset_index()

In [6]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_planet.hostname))
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1
        
masses_planet['Name'] = masses_table['Name']

In [6]:
namedict = {'10008':'hip7576'}

## Do the same for stellar binaries.

In [7]:
masses_table = pd.DataFrame()
#masses_table = masses_table.sort_values('P').sort_values('Name').reset_index()
masses_table['Name'] = masses_binary['hostname']
masses_table['Msini (MJup)'] = masses_binary['mass_med']
masses_table['a (AU)'] = masses_binary['axis_med']
masses_table['P (d)'] = masses_binary['per_med']
masses_table['e'] = masses_binary['e_med']

names = np.unique(np.array(masses_table.Name))

nbinarys = np.array(masses_table.groupby('Name').count()['e'])
db_nbinarys = pd.DataFrame({'Name':names, 'nbinarys':nbinarys})
masses_table = pd.merge(masses_table, db_nbinarys, on='Name').reset_index()

In [8]:
namedict = {0:' b', 1:' c', 2:' d', 3:' e', 4:' f', 5:' g', 6:' h', 7:' i'}

names = np.unique(np.array(masses_binary.hostname))
'''
for name in names:
    masses_name = masses_table.query('Name == "{}"'.format(name))
    numbah = len(masses_name)
    pl_index = 0
    for index in np.arange(numbah) + masses_name.index[0]:
        masses_table.loc[index, 'Name'] += namedict[pl_index]  
        pl_index += 1
'''     
masses_binary['Name'] = masses_table['Name'].str.upper()

In [9]:
def round_to_1(x):
    if np.isnan(x):
        return 0.0001
    else:
        return round(x, -int(floor(log10(x))))

def round_sig(x, sig=2):
    if x == np.nan:
        return np.nan
    else:
        return round(x, sig-int(floor(log10(abs(x)))) - 1) 
round_sig_vec = np.vectorize(round_sig)

def round_to_reference(x, y):
    if x == np.nan or y == np.nan:
        return np.nan
    else:
        return round(x, -int(floor(log10(abs(y)))) + 1)
round_to_ref_vec = np.vectorize(round_to_reference)

## Make table for planet parameters.

In [10]:
refdict = {'HD 145675 b': 'Wittenmyer07', 'HD 1326 b': 'Howard14',
           'HD 1461 b': 'Rivera10', 'HD 1461 c': 'Diaz16',
           'HD 155358 b': 'Robertson12', 'HD 155358 b': 'Stassun17',
           'HD 69830 b': 'Lovis06', 'HD 69830 c': 'Lovis06',
           'HD 69830 d': 'Lovis06', 'HD 102195 b': 'Ge06',
           'HD 104067 b': 'Segransan11', 'HD 10697 b': 'Butler06',
           'HD 107148 b': 'Butler06', 'HD 108874 b': 'Butler03',
           'HD 108874 c': 'Vogt05', 'HD 114729 b': 'Butler03',
           'HD 114762 b': 'Butler06', 'HD 114783 b': 'Vogt02',
           'HD 114783 c': 'Bryan16', 'HD 115617 b': 'Vogt10',
           'HD 115617 c': 'Vogt10', 'HD 115617 d': 'Vogt10',
           'HD 117176 b': 'Naef04', 'HD 117207 b': 'Marcy05',
           'HD 120136 b': 'Butler97', 'HD 12661 b': 'Fischer01',
           'HD 12661 c': 'Butler06', 'HD 126614 b': 'Howard10',
           'HD 128311 b': 'Butler03', 'HD 128311 c': 'Vogt05',
           'HD 130322 b': 'Udry00', 'HD 134987 b': 'Butler01',
           'HD 134987 c': 'Jones10', 'HD 13931 b': 'Howard10',
           'HD 141399 b': 'Vogt14', 'HD 141399 c': 'Vogt14',
           'HD 141399 d': 'Vogt14', 'HD 141399 e': 'Vogt14',
           'HD 143761 b': 'Noyes97', 'HD 143761 c': 'Fulton16',
           'HD 145934 b': 'Feng15', 'HD 154345 b': 'Wright08', 
           'HD 156279 b': 'Diaz12', 'HD 156279 c': 'Bryan16', 
           'HD 156668 b': 'Howard11','HD 16141 b': 'Butler06', 
           'HD 164922 b': 'Butler06','HD 164922 c': 'Fulton16',
           'HD 167042 b': 'Johnson08','HD 168443 b': 'Wittenmyer07',
           'HD 168443 c': 'Wittenmyer07','HD 168746 b': 'Pepe02',
           'HD 169830 b': 'Naef01','HD 169830 c': 'Mayor04',
           'HD 170469 b': 'Fischer07','HD 177830 b': 'Butler06',
           'HD 177830 c': 'Meschiari11','HD 179949 b': 'Tiney01',
           'HD 178911b b': 'Zucker02', 'HD 189733 b': 'Bouchy05',
           'HD 183263 b': 'Marcy05','HD 183263 c': 'Wright09',
           'HD 186427 b': 'Chochran97','HD 187123 b': 'Vogt00',
           'HD 187123 c': 'Wright09','HD 188015 b': 'Marcy05',
           'HD 190360 b': 'Naef03','HD 190360 c': 'Vogt05',
           'HD 192263 b': 'Santos00','HD 192310 b': 'Howard11',
           'HD 195019 b': 'Fischer99','HD 209458 b': 'Henry00',
           'HD 216520 b': 'Burt et al. in prep',
           'HD 210277 b': 'Marcy99','HD 217014 b': 'Mayor95',
           'HD 217107 b': 'Vogt05','HD 217107 c': 'Vogt05',
           'HD 218566 b': 'Meschiari11', 'HD 28185 b': 'Santos01', 
           'HD 285968 b':'Forveille09', 'HD 219134 b': 'Vogt15', 
           'HD 219134 c': 'Vogt15', 'HD 219134 d': 'Vogt15', 
           'HD 219134 e': 'Vogt15', 'HD 219134 f': 'Vogt15', 
           'HD 22049 b': 'Hatzes00','HD 222582 b': 'Vogt00',
           'HD 24040 b': 'Boisse12', 'HD 31253 b':'Meschiari11',
           'HD 32963 b': 'Rowan16', 'HD 90156 b':'Mordasini11',
           'HD 33636 b': 'Vogt02','HD 34445 b': 'Howard10',
           'HD 34445 c': 'Vogt17','HD 34445 d': 'Vogt17',
           'HD 34445 e': 'Vogt17', 'HD 3651 b': 'Butler06',
           'HD 38529 b': 'Fischer01','HD 38529 c': 'Butler06',
           'HD 40979 b': 'Butler06','HD 4203 b': 'Vogt02',
           'HD 4203 c': 'Kane14','HD 4208 b': 'Vogt02',
           'HD 42618 b': 'Fulton16','HD 45184 b': 'Udry19',
           'HD 45184 c': 'Udry19','HD 45350 b': 'Marcy05',
           'HD 46375 b': 'Marcy00','HD 49674 b': 'Butler03',
           'HD 50499 b': 'Vogt05','HD 50499 c': 'Rickman19',
           'HD 50554 b': 'Fischer02','HD 52265 b': 'Butler00',
           'HD 66428 b': 'Butler06','HD 72659 b': 'Butler03',
           'HD 74156 b': 'Naef04', 'HD 74156 c': 'Naef04',
           'HD 75732 b': 'Butler97','HD 75732 c': 'McArthur04',
           'HD 75732 d': 'Marcy02','HD 75732 e': 'McArthur04',
           'HD 75732 f': 'Fischer08', 'HD 7924 b': 'Fulton15',
           'HD 7924 c': 'Fulton15','HD 7924 d': 'Fulton15',
           'HD 82943 b': 'Mayor04','HD 82943 c': 'Mayor04',
           'HD 83443 b': 'Butler02','HD 8574 b': 'Perrier03',
           'HD 87883 b': 'Fischer09','HD 92788 b': 'Fischer01',
           'HD 92788 c': 'Rickman19','HD 9446 b': 'Hebrard10',
           'HD 9446 c': 'Hebrard10','HD 95128 b': 'Naef04',
           'HD 95128 c': 'Wittenmyer07', 'HD 95128 d': 'Gregory10',
           'HD 95735 b': 'Diaz19', 'HD 97101 b': 'Dedrick et al. in prep',
           'HD 97101 c': 'Dedrick et al. in prep',
           'HD 97658 b': 'Howard11','HD 9826 b': 'Butler97',
           'HD 9826 c': 'Naef04','HD 9826 d': 'Naef04',
           'HD 99109 b': 'Butler06', 'HD 147379a b': 'Reiners18',
           'HD 99492 b': 'Marcy05','HD 175541 b': 'Johnson07',
           'HD 181234 b': 'Rickman19', 'HD 11964a b': 'Butler06',
           'HD 11964a c': 'Wright09', 'GJ 3470 b': 'Bonfils12',
           'GL 317 b': 'Johnson07b', 'GL 317 c': 'Anglada-Escude12',
           'GL 687 b': 'Burt14', 'HIP 109388 b': 'Butler06_GJ849',
           'GL 876 b': 'Marcy01', 'GL 876 c': 'Marcy01', 
           'HIP 57087 b': 'Butler04','HIP 109388 c': 'Feng15', 
           'HIP 22627 b': 'Howard10','HIP 57050 b': 'Haghighipour10',
           'HIP 57050 c': 'Trifonov17','HIP 74995 b': 'Bonfils05', 
           'HIP 74995 c': 'Mayor09','HIP 83043 b': 'Johnson10',
           'BD-103166 b': 'Butler00','HD 37124 b': 'Butler03',
           'HD 37124 c': 'Butler03','HD 37124 d': 'Vogt05',
           'HD 68988 b': 'Vogt02','HD 80606 b': 'Wittenmyer07',
           'HD 120066 b': 'Blunt19'}

In [11]:
# Round errors and parameters with significant figures.
masses_planet['mass_err_plus_r']  = round_sig_vec(np.array(masses_planet['mass_plus'] - masses_planet['mass_med']))
masses_planet['mass_err_minus_r'] = round_sig_vec(np.array(masses_planet['mass_med'] - masses_planet['mass_minus']))
masses_planet['mass_err_av']      = round_sig_vec(0.5*(masses_planet['mass_err_plus_r'] + masses_planet['mass_err_minus_r']))
masses_planet['mass_med_r']       = round_to_ref_vec(np.array(masses_planet['mass_med']), masses_planet['mass_err_av'])

masses_planet['axis_err_plus_r']  = round_sig_vec(np.array(masses_planet['axis_plus'] - masses_planet['axis_med']))
masses_planet['axis_err_minus_r'] = round_sig_vec(np.array(masses_planet['axis_med'] - masses_planet['axis_minus']))
masses_planet['axis_err_av']      = round_sig_vec(0.5*(masses_planet['axis_err_plus_r'] + masses_planet['axis_err_minus_r']))
masses_planet['axis_med_r']       = round_to_ref_vec(np.array(masses_planet['axis_med']), masses_planet['axis_err_av'])

masses_planet['per_err_plus_r']   = round_sig_vec(np.array(masses_planet['per_plus'] - masses_planet['per_med']))
masses_planet['per_err_minus_r']  = round_sig_vec(np.array(masses_planet['per_med'] - masses_planet['per_minus']))
masses_planet['per_err_av']       = round_sig_vec(0.5*(masses_planet['per_err_plus_r'] + masses_planet['per_err_minus_r']))
masses_planet['per_med_r']        = round_to_ref_vec(np.array(masses_planet['per_med']), masses_planet['per_err_av'])

masses_planet['e_err_plus_r']     = round_sig_vec(np.array(masses_planet['e_plus'] - masses_planet['e_med']))
masses_planet['e_err_minus_r']    = round_sig_vec(np.array(masses_planet['e_med'] - masses_planet['e_minus']))
masses_planet['e_err_av']         = round_sig_vec(0.5*(masses_planet['e_err_plus_r'] + masses_planet['e_err_minus_r']))
masses_planet['e_med_r']          = round_to_ref_vec(np.array(masses_planet['e_med']), masses_planet['e_err_av'])

masses_planet['tc_err_plus_r']    = round_sig_vec(np.array(masses_planet['tc_plus'] - masses_planet['tc_med']))
masses_planet['tc_err_minus_r']   = round_sig_vec(np.array(masses_planet['tc_med'] - masses_planet['tc_minus']))
masses_planet['tc_err_av']        = round_sig_vec(0.5*(masses_planet['tc_err_plus_r'] + masses_planet['tc_err_minus_r']))
masses_planet['tc_med_r']         = round_to_ref_vec(np.array(masses_planet['tc_med'] - 2450000), masses_planet['tc_err_av'])

masses_planet['w_err_plus_r']     = round_sig_vec(np.array(masses_planet['w_plus'] - masses_planet['w_med']))
masses_planet['w_err_minus_r']    = round_sig_vec(np.array(masses_planet['w_med'] - masses_planet['w_minus']))
masses_planet['w_err_av']         = round_sig_vec(0.5*(masses_planet['w_err_plus_r'] + masses_planet['w_err_minus_r']))
masses_planet['w_med_r']          = round_to_ref_vec(np.array(masses_planet['w_med']), masses_planet['w_err_av'])

tex_writer = open('legacy_tables/tex_tables/planet_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Planet Catalog}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
#tex_writer.write(r'Name & \msini\ [\mjup] & $a$ [AU] & $e$ & $P$ [d] & $t_c - 2450000$ [d] & $\omega$ & Ref. \\' + ' \n')
#tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & $a$ [AU] & $e$ & $P$ [d] & Ref. \\' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & $a$ [AU] & $e$ & Ref. \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n') # $t_c - 2450000$ [d]
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Planet Catalog (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
#tex_writer.write(r'Name & \msini\ [\mjup] & $a$ [AU] & $e$ & $P$ [d] & $t_c - 2450000$ [d] & $\omega$ & Ref. \\' + ' \n')
#tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & $a$ [AU] & $e$ & $P$ [d] & Ref. \\' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & $a$ [AU] & $e$ & Ref. \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in masses_planet.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    # Make sure that uncertainties have the right number of significant figures, including final zeros.
    mass_str = str(row['mass_med_r'])
    mass_plus_str  = str(row['mass_err_plus_r'])
    mass_minus_str = str(row['mass_err_minus_r'])
    if len(mass_plus_str) < len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_plus_str += '0'
    elif len(mass_plus_str) > len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_minus_str += '0'    
    
    if mass_str[-2:] == '.0' and mass_plus_str[-2:] == '.0' and mass_minus_str[-2:] == '.0':
        mass_str = mass_str[:-2]
        mass_plus_str = mass_plus_str[:-2]
        mass_minus_str = mass_minus_str[:-2]
        
    axis_str = str(row['axis_med_r'])
    axis_plus_str  = str(row['axis_err_plus_r'])
    axis_minus_str = str(row['axis_err_minus_r'])
    if len(axis_plus_str) < len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_plus_str += '0'
    elif len(axis_plus_str) > len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_minus_str += '0' 
        
    if axis_str[-2:] == '.0' and axis_plus_str[-2:] == '.0' and axis_minus_str[-2:] == '.0':
        axis_str = axis_str[:-2]
        axis_plus_str = axis_plus_str[:-2]
        axis_minus_str = axis_minus_str[:-2]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    if per_str[-2:] == '.0' and per_plus_str[-2:] == '.0' and per_minus_str[-2:] == '.0':
        per_str = per_str[:-2]
        per_plus_str = per_plus_str[:-2]
        per_minus_str = per_minus_str[:-2]
        
    tc_str = str(row['tc_med_r'])
    tc_plus_str  = str(row['tc_err_plus_r'])
    tc_minus_str = str(row['tc_err_minus_r'])
    if len(tc_plus_str) < len(tc_minus_str) and (tc_plus_str[:2] == '0.' or tc_minus_str[:2] == '0.'):
        tc_plus_str += '0'
    elif len(tc_plus_str) > len(tc_minus_str) and (tc_plus_str[:2] == '0.' or tc_minus_str[:2] == '0.'):
        tc_minus_str += '0' 
        
    if tc_str[-2:] == '.0' and tc_plus_str[-2:] == '.0' and tc_minus_str[-2:] == '.0':
        tc_str = tc_str[:-2]
        tc_plus_str = tc_plus_str[:-2]
        tc_minus_str = tc_minus_str[:-2]
        
    w_str = str(row['w_med_r'])
    w_plus_str  = str(row['w_err_plus_r'])
    w_minus_str = str(row['w_err_minus_r'])
    if len(w_plus_str) < len(w_minus_str) and (w_plus_str[:2] == '0.' or w_minus_str[:2] == '0.'):
        w_plus_str += '0'
    elif len(w_plus_str) > len(w_minus_str) and (w_plus_str[:2] == '0.' or w_minus_str[:2] == '0.'):
        w_minus_str += '0' 
        
    if w_str[-2:] == '.0' and w_plus_str[-2:] == '.0' and w_minus_str[-2:] == '.0':
        w_str = w_str[:-2]
        w_plus_str = w_plus_str[:-2]
        w_minus_str = w_minus_str[:-2]
    
    #if row['e_mode'] > row['e_plus'] - row['e_minus']:
    ecc_str = str(row['e_med_r'])
    ecc_plus_str  = str(row['e_err_plus_r'])
    ecc_minus_str = str(row['e_err_minus_r'])
    if len(ecc_plus_str) < len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_plus_str += '0'
    elif len(ecc_plus_str) > len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_minus_str += '0'
    #else:
    #    ecc_str = '[0, {}]'.format(row['e_68'])
        
    # Solve SI issues, for period.
    if 'e' in per_str and 'e' in per_plus_str:
        meanbase, meanexp   = str.split(per_str, sep='e')
        if meanexp[1] == '0':
            meanexp = '-' + meanexp[-1]
        plusbase, plusexp = str.split(per_plus_str, sep='e')
        if plusexp[1] == '0':
            plusexp = '-' + plusexp[-1]
        minusbase, minusexp = str.split(per_minus_str, sep='e')
        if minusexp[1] == '0':
            minusexp =  '-' + minusexp[-1]
        per_str       = meanbase + '*10^{' + meanexp + '}'
        per_plus_str  = plusbase + '*10^{' + plusexp + '}'
        per_minus_str = minusbase + '*10^{' + minusexp + '}'  
 
    elif 'e' in per_plus_str and (not 'e' in per_str):
        plusbase, plusexp   = str.split(per_plus_str, sep='e')
        if plusexp[1] == '0':
            plusexp = '-' + plusexp[-1]
        minusbase, minusexp = str.split(per_minus_str, sep='e')
        if minusexp[1] == '0':
            minusexp = '-' + minusexp[-1]
        per_plus_str  = plusbase + '*10^{' + plusexp + '}'
        per_minus_str = minusbase + '*10^{' + minusexp + '}'  
        
    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(mass_str, mass_plus_str, mass_minus_str) + r'$'    
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(axis_str, axis_plus_str, axis_minus_str) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    #if row['e_mode'] > row['e_plus'] - row['e_minus']:
    #    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    #else:
    #    eccein = ecc_str
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(tc_str, tc_plus_str, tc_minus_str) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(w_str, w_plus_str, w_minus_str) + r'$'
    
    if namein in refdict.keys():
        ref = '\cite{' + refdict[namein] + '}'
    else:
        ref = 'This work'

    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} & \\\\ \n'.format(
                     namein, namein, massin, axisin, eccein, ref))
    #tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
    #                 namein, massin, axisin, eccein, periin, '\cite'))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make new candidate table.

In [12]:
masses_candidate = masses_planet.query('status == "C" or status == "J"').reset_index(drop=True)

# Round errors and parameters with significant figures.
masses_candidate['mass_err_plus_r']  = round_sig_vec(np.array(masses_candidate['mass_plus'] - masses_candidate['mass_med']))
masses_candidate['mass_err_minus_r'] = round_sig_vec(np.array(masses_candidate['mass_med'] - masses_candidate['mass_minus']))
masses_candidate['mass_err_av']      = round_sig_vec(0.5*(masses_candidate['mass_err_plus_r'] + masses_candidate['mass_err_minus_r']))
masses_candidate['mass_med_r']       = round_to_ref_vec(np.array(masses_candidate['mass_med']), masses_candidate['mass_err_av'])

masses_candidate['earth_err_plus_r']  = round_sig_vec(317.8*np.array(masses_candidate['mass_plus'] - masses_candidate['mass_med']))
masses_candidate['earth_err_minus_r'] = round_sig_vec(317.8*np.array(masses_candidate['mass_med'] - masses_candidate['mass_minus']))
masses_candidate['earth_err_av']      = round_sig_vec(0.5*(masses_candidate['earth_err_plus_r'] + masses_candidate['earth_err_minus_r']))
masses_candidate['earth_med_r']       = round_to_ref_vec(np.array(317.8*masses_candidate['mass_med']), masses_candidate['earth_err_av'])

masses_candidate['axis_err_plus_r']  = round_sig_vec(np.array(masses_candidate['axis_plus'] - masses_candidate['axis_med']))
masses_candidate['axis_err_minus_r'] = round_sig_vec(np.array(masses_candidate['axis_med'] - masses_candidate['axis_minus']))
masses_candidate['axis_err_av']      = round_sig_vec(0.5*(masses_candidate['axis_err_plus_r'] + masses_candidate['axis_err_minus_r']))
masses_candidate['axis_med_r']       = round_to_ref_vec(np.array(masses_candidate['axis_med']), masses_candidate['axis_err_av'])

masses_candidate['per_err_plus_r']   = round_sig_vec(np.array(masses_candidate['per_plus'] - masses_candidate['per_med']))
masses_candidate['per_err_minus_r']  = round_sig_vec(np.array(masses_candidate['per_med'] - masses_candidate['per_minus']))
masses_candidate['per_err_av']       = round_sig_vec(0.5*(masses_candidate['per_err_plus_r'] + masses_candidate['per_err_minus_r']))
masses_candidate['per_med_r']        = round_to_ref_vec(np.array(masses_candidate['per_med']), masses_candidate['per_err_av'])

masses_candidate['e_err_plus_r']     = round_sig_vec(np.array(masses_candidate['e_plus'] - masses_candidate['e_med']))
masses_candidate['e_err_minus_r']    = round_sig_vec(np.array(masses_candidate['e_med'] - masses_candidate['e_minus']))
masses_candidate['e_err_av']         = round_sig_vec(0.5*(masses_candidate['e_err_plus_r'] + masses_candidate['e_err_minus_r']))
masses_candidate['e_med_r']          = round_to_ref_vec(np.array(masses_candidate['e_med']), masses_candidate['e_err_av'])

masses_candidate['tc_err_plus_r']    = round_sig_vec(np.array(masses_candidate['tc_plus'] - masses_candidate['tc_med']))
masses_candidate['tc_err_minus_r']   = round_sig_vec(np.array(masses_candidate['tc_med'] - masses_candidate['tc_minus']))
masses_candidate['tc_err_av']        = round_sig_vec(0.5*(masses_candidate['tc_err_plus_r'] + masses_candidate['tc_err_minus_r']))
masses_candidate['tc_med_r']         = round_to_ref_vec(np.array(masses_candidate['tc_med'] - 2450000), masses_candidate['tc_err_av'])

masses_candidate['w_err_plus_r']     = round_sig_vec(np.array(masses_candidate['w_plus'] - masses_candidate['w_med']))
masses_candidate['w_err_minus_r']    = round_sig_vec(np.array(masses_candidate['w_med'] - masses_candidate['w_minus']))
masses_candidate['w_err_av']         = round_sig_vec(0.5*(masses_candidate['w_err_plus_r'] + masses_candidate['w_err_minus_r']))
masses_candidate['w_med_r']          = round_to_ref_vec(np.array(masses_candidate['w_med']), masses_candidate['w_err_av'])

tex_writer = open('legacy_tables/tex_tables/candidate_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Newly Discovered and Revised Planets and Substellar Companions}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] (\mearth) & $a$ [AU] & $e$ & $P$ [d] & $t_c - 2450000$ [d] & $\omega$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')

for index, row in masses_candidate.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    # Make sure that uncertainties have the right number of significant figures, including final zeros.
    mass_str = str(row['mass_med_r'])
    mass_plus_str  = str(row['mass_err_plus_r'])
    mass_minus_str = str(row['mass_err_minus_r'])
    if len(mass_plus_str) < len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_plus_str += '0'
    elif len(mass_plus_str) > len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_minus_str += '0'    
    
    if mass_str[-2:] == '.0' and mass_plus_str[-2:] == '.0' and mass_minus_str[-2:] == '.0':
        mass_str = mass_str[:-2]
        mass_plus_str = mass_plus_str[:-2]
        mass_minus_str = mass_minus_str[:-2]
        
    earth_str = str(row['earth_med_r'])
    earth_plus_str  = str(row['earth_err_plus_r'])
    earth_minus_str = str(row['earth_err_minus_r'])
    if len(earth_plus_str) < len(earth_minus_str) and (earth_plus_str[:2] == '0.' or earth_minus_str[:2] == '0.'):
        earth_plus_str += '0'
    elif len(earth_plus_str) > len(earth_minus_str) and (earth_plus_str[:2] == '0.' or earth_minus_str[:2] == '0.'):
        earth_minus_str += '0'    
    
    if earth_str[-2:] == '.0' and earth_plus_str[-2:] == '.0' and earth_minus_str[-2:] == '.0':
        earth_str = earth_str[:-2]
        earth_plus_str = earth_plus_str[:-2]
        earth_minus_str = earth_minus_str[:-2]
        
    axis_str = str(row['axis_med_r'])
    axis_plus_str  = str(row['axis_err_plus_r'])
    axis_minus_str = str(row['axis_err_minus_r'])
    if len(axis_plus_str) < len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_plus_str += '0'
    elif len(axis_plus_str) > len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_minus_str += '0' 
        
    if axis_str[-2:] == '.0' and axis_plus_str[-2:] == '.0' and axis_minus_str[-2:] == '.0':
        axis_str = axis_str[:-2]
        axis_plus_str = axis_plus_str[:-2]
        axis_minus_str = axis_minus_str[:-2]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    if per_str[-2:] == '.0' and per_plus_str[-2:] == '.0' and per_minus_str[-2:] == '.0':
        per_str = per_str[:-2]
        per_plus_str = per_plus_str[:-2]
        per_minus_str = per_minus_str[:-2]

    tc_str = str(row['tc_med_r'])
    tc_plus_str  = str(row['tc_err_plus_r'])
    tc_minus_str = str(row['tc_err_minus_r'])
    if len(tc_plus_str) < len(tc_minus_str) and (tc_plus_str[:2] == '0.' or tc_minus_str[:2] == '0.'):
        tc_plus_str += '0'
    elif len(tc_plus_str) > len(tc_minus_str) and (tc_plus_str[:2] == '0.' or tc_minus_str[:2] == '0.'):
        tc_minus_str += '0' 
        
    if tc_str[-2:] == '.0' and tc_plus_str[-2:] == '.0' and tc_minus_str[-2:] == '.0':
        tc_str = tc_str[:-2]
        tc_plus_str = tc_plus_str[:-2]
        tc_minus_str = tc_minus_str[:-2]
        
    w_str = str(row['w_med_r'])
    w_plus_str  = str(row['w_err_plus_r'])
    w_minus_str = str(row['w_err_minus_r'])
    if len(w_plus_str) < len(w_minus_str) and (w_plus_str[:2] == '0.' or w_minus_str[:2] == '0.'):
        w_plus_str += '0'
    elif len(w_plus_str) > len(w_minus_str) and (w_plus_str[:2] == '0.' or w_minus_str[:2] == '0.'):
        w_minus_str += '0' 
        
    if w_str[-2:] == '.0' and w_plus_str[-2:] == '.0' and w_minus_str[-2:] == '.0':
        w_str = w_str[:-2]
        w_plus_str = w_plus_str[:-2]
        w_minus_str = w_minus_str[:-2]        
        
    ecc_str = str(row['e_med_r'])
    ecc_plus_str  = str(row['e_err_plus_r'])
    ecc_minus_str = str(row['e_err_minus_r'])
    if len(ecc_plus_str) < len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_plus_str += '0'
    elif len(ecc_plus_str) > len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_minus_str += '0' 
        
    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(mass_str, mass_plus_str, mass_minus_str) + r'$'
    earthin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(earth_str, earth_plus_str, earth_minus_str) + r'$'    
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(axis_str, axis_plus_str, axis_minus_str) + r'$'    
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(tc_str, tc_plus_str, tc_minus_str) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(w_str, w_plus_str, w_minus_str) + r'$'
    
    tex_writer.write('{0} & {1} & {2} ({3}) & {4} & {5} & {6} & {7} & {8} \\\\ \n'.format(
                     namein, namein, massin, earthin, axisin, eccein, periin, tcin, win))  
    #tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
    #                 namein, massin, axisin, periin, eccein))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make table for stellar parameters. 

In [13]:
# Specmatch parameters first. TO-DO: COMBINE SYNTHETIC/EMPIRICIAL RESULTS. MAKE COLUMN WITH SYN/EMP
stellar['teff_err_r'] = round_sig_vec(stellar['teff_err_c'])
stellar['teff_r'] = round_to_ref_vec(np.array(stellar['teff_c']), stellar['teff_err_r'])
stellar['fe_err_r'] = round_sig_vec(stellar['fe_err_c'])
stellar['fe_r'] = round_to_ref_vec(np.array(stellar['fe_c']), stellar['fe_err_r'])

# USE ISOCLASSIFY OUTPUTS FOR R AND LOGG, SINCE SPECMATCH-SYN AND -EMP EACH ONLY PRODUCE ONE. 
# FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX FIX
stellar['logg_err_r'] = round_sig_vec(stellar['logg_err_c'])
stellar['logg_r'] = round_to_ref_vec(np.array(stellar['logg_c']), stellar['logg_err_r'])
stellar['radius_err_r'] = round_sig_vec(stellar['radius_err_c'])
stellar['radius_r'] = round_to_ref_vec(np.array(stellar['radius_c']), stellar['radius_err_r'])

# Now isoclassify.
stellar['mass_err_r'] = round_sig_vec(stellar['mass_err_c'])
stellar['mass_r'] = round_to_ref_vec(np.array(stellar['mass_c']), stellar['mass_err_r'])


tex_writer = open('legacy_tables/tex_tables/stellar_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Stellar Catalog}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & $T_{\textrm{eff}}$ [K] & [Fe/H] & log $g$ [log(cm s$^{-2}$)] & $R$ [\rsun] & $M$ [\msun] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Stellar Catalog (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'Name & $T_{\textrm{eff}}$ [K] & [Fe/H] & log $g$ [log(cm s$^{-2}$)] & $R$ [\rsun] & $M$ [\msun] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in stellar.iterrows():
    # Customize names.
    namein = str(row['name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    teff_str   = r'$' + '{0}\\pm {1}'.format(row['teff_r'], row['teff_err_r']) + r'$'    
    fe_str     = r'$' + '{0}\\pm {1}'.format(row['fe_r'], row['fe_err_r']) + r'$'    
    logg_str   = r'$' + '{0}\\pm {1}'.format(row['logg_r'], row['logg_err_r']) + r'$' 
    radius_str = r'$' + '{0}\\pm {1}'.format(row['radius_r'], row['radius_err_r']) + r'$'      
    mass_str   = r'$' + '{0}\\pm {1}'.format(row['mass_r'], row['mass_err_r']) + r'$'

    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, teff_str, fe_str, logg_str, radius_str, mass_str))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

ValueError: math domain error

## Make false-positive table.

In [10]:
false_positive['per_err_plus_r']   = round_sig_vec(np.array(false_positive['per_plus'] - false_positive['per_med']))
false_positive['per_err_minus_r']  = round_sig_vec(np.array(false_positive['per_med'] - false_positive['per_minus']))
false_positive['per_err_av']       = round_sig_vec(0.5*(false_positive['per_err_plus_r'] + false_positive['per_err_minus_r']))
false_positive['per_med_r']        = round_to_ref_vec(np.array(false_positive['per_med']), false_positive['per_err_av'])

false_positive['k_err_plus_r']   = round_sig_vec(np.array(false_positive['k_plus'] - false_positive['k_med']))
false_positive['k_err_minus_r']  = round_sig_vec(np.array(false_positive['k_med'] - false_positive['k_minus']))
false_positive['k_err_av']       = round_sig_vec(0.5*(false_positive['k_err_plus_r'] + false_positive['k_err_minus_r']))
false_positive['k_med_r']        = round_to_ref_vec(np.array(false_positive['k_med']), false_positive['k_err_av'])

false_positive['k_r'] = round_sig_vec(np.array(false_positive['k_med']))

tex_writer = open('legacy_tables/tex_tables/false_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{False Positives}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'CPS Name & Lit. Name & $P$ [d] & $K$ [m s$^{-1}$] & Cause \\' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{False Positives (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & $P$ [d] & $K$ [m s$^{-1}$] & Cause \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in false_positive.iterrows():
    # Customize names.
    namein = str(row['hostname'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein[:]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    k_str = str(row['k_med_r'])
    k_plus_str  = str(row['k_err_plus_r'])
    k_minus_str = str(row['k_err_minus_r'])
    if len(k_plus_str) < len(k_minus_str) and (k_plus_str[:2] == '0.' or k_minus_str[:2] == '0.'):
        k_plus_str += '0'
    elif len(k_plus_str) > len(k_minus_str) and (k_plus_str[:2] == '0.' or k_minus_str[:2] == '0.'):
        k_minus_str += '0' 

    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    kin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(k_str, k_plus_str, k_minus_str) + r'$'
    reason = str(row['status'])

    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, namein, periin, kin, reason))    
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make stellar/substellar binary table.

In [14]:
masses_binary = masses_binary.fillna(10.5)
masses_binary = masses_binary.query('mass_med > 0 and mass_minus > 0 and mass > 0 and \
                                     k_med > 0 and k_minus > 0 and k > 0 and \
                                     axis_med > 0 and axis_minus > 0 and axis > 0 and \
                                     per_med > 0 and per_minus > 0 and per > 0 and \
                                     hostname != "3130-1591-1"').reset_index()

In [15]:
masses_binary[['hostname', 'axis', 'axis_minus', 'axis_med', 'axis_plus']]

,hostname,axis,axis_minus,axis_med,axis_plus
0,104304,14.085016,16.491101,18.057896,19.476732
1,10790,1.544500,1.523955,1.544396,1.564648
2,111031,21.873947,22.185986,32.220638,47.349448
3,112914,1.388567,1.372346,1.388473,1.404202
4,114762,0.352046,0.349429,0.352049,0.354624
5,120136,33.611088,28.258167,46.241494,92.094007
6,126614,14.920767,14.985698,16.495014,18.152885
7,142229,13.656286,16.994624,27.690948,43.762840
8,142267,0.492420,0.487440,0.492406,0.497262
9,144287,5.056744,4.991217,5.043927,5.095128


In [16]:
masses_binary['mass_err_plus_r']  = round_sig_vec(np.abs(np.array(masses_binary['mass_plus'] - masses_binary['mass_med'])))
masses_binary['mass_err_minus_r'] = round_sig_vec(np.abs(np.array(masses_binary['mass_med'] - masses_binary['mass_minus'])))
masses_binary['mass_err_av']      = round_sig_vec(0.5*(masses_binary['mass_err_plus_r'] + masses_binary['mass_err_minus_r']))
masses_binary['mass_med_r']       = round_to_ref_vec(np.array(masses_binary['mass_med']), masses_binary['mass_err_av'])

masses_binary['axis_err_plus_r']  = round_sig_vec(np.abs(np.array(masses_binary['axis_plus'] - masses_binary['axis_med'])))
masses_binary['axis_err_minus_r'] = round_sig_vec(np.abs(np.array(masses_binary['axis_med'] - masses_binary['axis_minus'])))
masses_binary['axis_err_av']      = round_sig_vec(0.5*(masses_binary['axis_err_plus_r'] + masses_binary['axis_err_minus_r']))
masses_binary['axis_med_r']       = round_to_ref_vec(np.array(masses_binary['axis_med']), masses_binary['axis_err_av'])

masses_binary['per_err_plus_r']   = round_sig_vec(np.abs(np.array(masses_binary['per_plus'] - masses_binary['per_med'])))
masses_binary['per_err_minus_r']  = round_sig_vec(np.abs(np.array(masses_binary['per_med'] - masses_binary['per_minus'])))
masses_binary['per_err_av']       = round_sig_vec(0.5*(masses_binary['per_err_plus_r'] + masses_binary['per_err_minus_r']))
masses_binary['per_med_r']        = round_to_ref_vec(np.array(masses_binary['per_med']), masses_binary['per_err_av'])

masses_binary['e_err_plus_r']     = round_sig_vec(np.abs(np.array(masses_binary['e_plus'] - masses_binary['e_med'])))
masses_binary['e_err_minus_r']    = round_sig_vec(np.abs(np.array(masses_binary['e_med'] - masses_binary['e_minus'])))
masses_binary['e_err_av']         = round_sig_vec(0.5*(masses_binary['e_err_plus_r'] + masses_binary['e_err_minus_r']))
masses_binary['e_med_r']          = round_to_ref_vec(np.array(masses_binary['e_med']), masses_binary['e_err_av'])

masses_binary['tc_err_plus_r']     = round_sig_vec(np.array(masses_binary['tc_plus'] - masses_binary['tc_med']))
masses_binary['tc_err_minus_r']    = round_sig_vec(np.array(masses_binary['tc_med'] - masses_binary['tc_minus']))
masses_binary['tc_err_av']         = round_sig_vec(0.5*(masses_binary['tc_err_plus_r'] + masses_binary['tc_err_minus_r']))
masses_binary['tc_med_r']          = round_to_ref_vec(np.array(masses_binary['tc_med'] - 2450000), masses_binary['tc_err_av'])

masses_binary['w_err_plus_r']     = round_sig_vec(np.array(masses_binary['w_plus'] - masses_binary['w_med']))
masses_binary['w_err_minus_r']    = round_sig_vec(np.array(masses_binary['w_med'] - masses_binary['w_minus']))
masses_binary['w_err_av']         = round_sig_vec(0.5*(masses_binary['w_err_plus_r'] + masses_binary['w_err_minus_r']))
masses_binary['w_med_r']          = round_to_ref_vec(np.array(masses_binary['w_med']), masses_binary['w_err_av'])

tex_writer = open('legacy_tables/tex_tables/binary_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Binary and Substellar Catalog}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
#tex_writer.write(r'Name & \msini\ [\mjup] & $a$ [AU] & $e$ & $P$ [d] & $t_c - 2450000$ [d] & $\omega$ \\' + ' \n')
#tex_writer.write(r'Name & \msini\ [\mjup] & $a$ [AU] & $e$ \\' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & $a$ [AU] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Binary and Substellar Catalog (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
#tex_writer.write(r'Name & \msini\ [\mjup] & $a$ [AU] & $e$ & $P$ [d] & $t_c - 2450000$ [d] & $\omega$ \\' + ' \n')
#tex_writer.write(r'Name & \msini\ [\mjup] & $a$ [AU] & $e$ \\' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & \msini\ [\mjup] & $a$ [AU] & $e$ \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in masses_binary.iterrows():
    # Customize names.
    namein = str(row['Name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:]
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:]
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:]
    else:
        namein = 'HD ' + namein

    # Make sure that uncertainties have the right number of significant figures, including final zeros.
    mass_str = str(row['mass_med_r'])
    mass_plus_str  = str(row['mass_err_plus_r'])
    mass_minus_str = str(row['mass_err_minus_r'])
    if len(mass_plus_str) < len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_plus_str += '0'
    elif len(mass_plus_str) > len(mass_minus_str) and (mass_plus_str[:2] == '0.' or mass_minus_str[:2] == '0.'):
        mass_minus_str += '0'    
    
    if mass_str[-2:] == '.0' and mass_plus_str[-2:] == '.0' and mass_minus_str[-2:] == '.0':
        mass_str = mass_str[:-2]
        mass_plus_str = mass_plus_str[:-2]
        mass_minus_str = mass_minus_str[:-2]
        
    axis_str = str(row['axis_med_r'])
    axis_plus_str  = str(row['axis_err_plus_r'])
    axis_minus_str = str(row['axis_err_minus_r'])
    if len(axis_plus_str) < len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_plus_str += '0'
    elif len(axis_plus_str) > len(axis_minus_str) and (axis_plus_str[:2] == '0.' or axis_minus_str[:2] == '0.'):
        axis_minus_str += '0' 
        
    if axis_str[-2:] == '.0' and axis_plus_str[-2:] == '.0' and axis_minus_str[-2:] == '.0':
        axis_str = axis_str[:-2]
        axis_plus_str = axis_plus_str[:-2]
        axis_minus_str = axis_minus_str[:-2]
        
    per_str = str(row['per_med_r'])
    per_plus_str  = str(row['per_err_plus_r'])
    per_minus_str = str(row['per_err_minus_r'])
    if len(per_plus_str) < len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_plus_str += '0'
    elif len(per_plus_str) > len(per_minus_str) and (per_plus_str[:2] == '0.' or per_minus_str[:2] == '0.'):
        per_minus_str += '0' 
        
    if per_str[-2:] == '.0' and per_plus_str[-2:] == '.0' and per_minus_str[-2:] == '.0':
        per_str = per_str[:-2]
        per_plus_str = per_plus_str[:-2]
        per_minus_str = per_minus_str[:-2]

    tc_str = str(row['tc_med_r'])
    tc_plus_str  = str(row['tc_err_plus_r'])
    tc_minus_str = str(row['tc_err_minus_r'])
    if len(tc_plus_str) < len(tc_minus_str) and (tc_plus_str[:2] == '0.' or tc_minus_str[:2] == '0.'):
        tc_plus_str += '0'
    elif len(tc_plus_str) > len(tc_minus_str) and (tc_plus_str[:2] == '0.' or tc_minus_str[:2] == '0.'):
        tc_minus_str += '0' 
        
    if tc_str[-2:] == '.0' and tc_plus_str[-2:] == '.0' and tc_minus_str[-2:] == '.0':
        tc_str = tc_str[:-2]
        tc_plus_str = tc_plus_str[:-2]
        tc_minus_str = tc_minus_str[:-2]
        
    w_str = str(row['w_med_r'])
    w_plus_str  = str(row['w_err_plus_r'])
    w_minus_str = str(row['w_err_minus_r'])
    if len(w_plus_str) < len(w_minus_str) and (w_plus_str[:2] == '0.' or w_minus_str[:2] == '0.'):
        w_plus_str += '0'
    elif len(w_plus_str) > len(w_minus_str) and (w_plus_str[:2] == '0.' or w_minus_str[:2] == '0.'):
        w_minus_str += '0' 
        
    if w_str[-2:] == '.0' and w_plus_str[-2:] == '.0' and w_minus_str[-2:] == '.0':
        w_str = w_str[:-2]
        w_plus_str = w_plus_str[:-2]
        w_minus_str = w_minus_str[:-2]        
        
    ecc_str = str(row['e_med_r'])
    ecc_plus_str  = str(row['e_err_plus_r'])
    ecc_minus_str = str(row['e_err_minus_r'])
    if len(ecc_plus_str) < len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_plus_str += '0'
    elif len(ecc_plus_str) > len(ecc_minus_str) and (ecc_plus_str[:2] == '0.' or ecc_minus_str[:2] == '0.'):
        ecc_minus_str += '0' 
        
    massin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(mass_str, mass_plus_str, mass_minus_str) + r'$'    
    axisin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(axis_str, axis_plus_str, axis_minus_str) + r'$'    
    periin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(per_str, per_plus_str, per_minus_str) + r'$'
    eccein = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(ecc_str, ecc_plus_str, ecc_minus_str) + r'$'
    tcin = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(tc_str, tc_plus_str, tc_minus_str) + r'$'
    win = r'$' + '{0}^{{+{1}}}_{{-{2}}}'.format(w_str, w_plus_str, w_minus_str) + r'$'
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} \\\\ \n'.format(
                     namein, namein, massin, axisin, eccein))#, periin, tcin, win))  
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

## Make table for trend and curvature terms.

In [15]:
trends_curv = trends.query('curv == curv')
#pdb.set_trace()

# Round errors and parameters with significant figures.
trends['dvdt_err_plus_r']  = round_sig_vec(np.array(trends['dvdt_plus'] - trends['dvdt_med']))
trends['dvdt_err_minus_r'] = round_sig_vec(np.array(trends['dvdt_med'] - trends['dvdt_minus']))
trends['dvdt_err_av']      = round_sig_vec(0.5*(trends['dvdt_err_plus_r'] + trends['dvdt_err_minus_r']))
trends['dvdt_med_r']       = round_to_ref_vec(np.array(trends['dvdt_med']), trends['dvdt_err_av'])

# Combine curvature and non-curvature databases.
trends_curv['curv_err_plus_r']  = round_sig_vec(np.array(trends_curv['curv_plus'] - trends_curv['curv_med']))
trends_curv['curv_err_minus_r'] = round_sig_vec(np.array(trends_curv['curv_med'] - trends_curv['curv_minus']))
trends_curv['curv_err_av']      = round_sig_vec(0.5*(trends_curv['curv_err_plus_r'] + trends_curv['curv_err_minus_r']))
trends_curv['curv_med_r']       = round_to_ref_vec(np.array(trends_curv['curv_med']), trends_curv['curv_err_av'])

#trends = trends.merge(trends_curv, on='hostname', suffixes=('', '_curv'))
#trends = trends.update(trends_curv)
#pdb.set_trace()

tex_writer = open('legacy_tables/tex_tables/trends_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{llrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Long Term Trends}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'CPS Name & Lit. Name & $\dot \gamma$ [m s$^{-1}$ d$^{-1}$] & $\ddot \gamma$ [m s$^{-1}$ d$^{-2}$] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Long Term Trends (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'CPS Name & Lit. Name & $\dot \gamma$ [m s$^{-1}$ d$^{-1}$] & $\ddot \gamma$ [m s$^{-1}$ d$^{-2}$] \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in trends.iterrows():
    # Customize names.
    basename = str(row['hostname'])
    namein = str(row['hostname'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:].upper()
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:].upper()
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:].upper()
    else:
        namein = 'HD ' + namein.upper()

    dvdtin = r'$' + '{0}\\pm {1}'.format(row['dvdt_med_r'], 
                                         row['dvdt_err_av']) + r'$'
    
    if basename in trends_curv['hostname'].tolist():
        row = trends_curv.query('hostname == "{}"'.format(basename))
        meanstr = str(row['curv_med_r'].to_list()[0])
        errstr  = str(row['curv_err_av'].to_list()[0])

        if 'e' in meanstr and 'e' in errstr:
            meanbase, meanexp = str.split(meanstr, sep='e')
            errbase, errexp = str.split(errstr, sep='e')
            curvin = r'$' + '{0}*10^{1}\\pm {2}*10^{3}'.format(meanbase, 
                                                               '{-' + meanexp[-1] + '}', errbase, 
                                                               '{-' + errexp[-1]) + '}' + r'$'
        elif 'e' in errstr and (not 'e' in meanstr):
            errbase, errexp = str.split(errstr, sep='e')
            curvin = r'$' + '{0}\\pm {1}*10^{2}'.format(meanstr, errbase, 
                                                        '{-' + errexp[-1]) + '}' + r'$'
        elif 'e' in meanstr and (not 'e' in errstr):
            meanbase, meanexp = str.split(meanstr, sep='e')
            curvin = r'$' + '{0}*10^{1}\\pm {2}'.format(meanbase, 
                                                               '{-' + meanexp[-1] + '}', 
                                                                 errstr) + r'$'
        else:
            curvin = r'$' + '{0}\\pm {1}'.format(meanstr, errstr) + r'$'    
    else:
        curvin = '0'

    
    tex_writer.write('{0} & {1} & {2} & {3} \\\\ \n'.format(namein, namein, dvdtin, curvin))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

In [11]:
data.columns

Index(['Unnamed: 0', 'JD_UTC', 'OBSNM', 'Prot', 'S-value', 'S/N', 'SVAL',
       'age', 'bc', 'comment', 'cts', 'date', 'date_sval', 'day', 'day_sval',
       'decker', 'dewar', 'errvel', 'gain', 'hjd_utc', 'hjd_utc_sval', 'inst',
       'iodnm', 'jd', 'jd_sval', 'jitter', 'kjd', 'kjd_sval', 'mdchi', 'mdpar',
       'mdvel', 'med_all', 'mnpar', 'mnvel', 'name', 'nchunk', 'obnm', 'obs',
       'phase', 'psfpix', 'psfsig', 'rphk', 'sn', 'sp1', 'sp2', 'spst', 'star',
       'sval', 'tel', 'z'],
      dtype='object')

In [46]:
data_select = data[['name', 'tel', 'obnm', 'jd', 'mnvel', 'errvel', 'sval', 'cts']]

In [13]:
data_sample = data_select.query('name == "156668"')[:13]

In [14]:
teldict = {'k':'HIRES pre-2004', 'j':'HIRES post-2004 ', 'apf':'APF', 'lick':'Lick'}

tex_writer = open('legacy_tables/tex_tables/data_table.tex', 'w')
tex_writer.write(r'\begin{longtable*}{lrrrrr}')
tex_writer.write('\n')
tex_writer.write(r'\caption{Sample of RV Data}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write('\n')
tex_writer.write(r'Name & Inst. & BJD - 2455000 & $RV$ [m/s] & $\sigma_{RV}$ [m/s] & S-Value \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endfirsthead' + ' \n')
tex_writer.write(r'\caption[]{Sample of RV Data (Continued)}' + r' \\')
tex_writer.write('\n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\midrule' + ' \n')
tex_writer.write(r'Name & Inst. & BJD - 2455000 & $RV$ [m/s] & $\sigma_{RV}$ [m/s] & S-Value \\' + ' \n')
tex_writer.write(r'\toprule' + ' \n')
tex_writer.write(r'\endhead' + ' \n')

for index, row in data_sample.iterrows():
    # Customize names.
    namein = str(row['name'])
    if namein[0] == 'h':
        namein = namein[:3].upper() + ' ' + namein[3:].upper()
    elif namein[0] == 'g':
        namein = namein[:2].upper() + ' ' + namein[2:].upper()
    elif namein[0] == 'b':
        namein = namein[:2].upper() + namein[2:].upper()
    elif namein[0] == 's':
        namein = namein.upper()
    else:
        namein = 'HD ' + namein.upper()

    telstr    = teldict[row['tel']]
    jdstr     = str(round(row['jd'] - 2455000, 4))
    mnvelstr  = str(round(row['mnvel'], 2))
    errvelstr = str(round(row['errvel'], 2))
    svalstr   = str(round(row['sval'], 3))
    
    tex_writer.write('{0} & {1} & {2} & {3} & {4} & {5} \\\\ \n'.format(
                     namein, telstr, jdstr, mnvelstr, errvelstr, svalstr))
    
tex_writer.write(r'\bottomrule' + ' \n')
tex_writer.write(r'\end{longtable*}' + ' \n')
tex_writer.close()

In [13]:
len(data)

103991

## Make rounded versions of the tables, and include planet citations.

In [11]:
stellar.columns

Index(['Unnamed: 0', 'name', 'teff_c', 'teff_err_c', 'fe_c', 'fe_err_c',
       'logg_c', 'logg_err_c', 'radius_c', 'radius_err_c', 'mass_c',
       'mass_err_c', 'star', 'mv', 'bvcolor', 'parallax'],
      dtype='object')

In [12]:
s_round_dict = {'teff_c':1, 'teff_err_c':1, 'fe_c':3, 'fe_err_c':3, 
                'logg_c':3, 'logg_err_c':3, 'radius_c':3, 'radius_err_c':3, 
                'mass_c':3, 'mass_err_c':3, 'mv':4, 'bvcolor':4, 'parallax':5}
stellar_rounded = stellar.round(s_round_dict)[['name', 'teff_c', 'teff_err_c', 'fe_c', 'fe_err_c',
                                               'logg_c', 'logg_err_c', 'radius_c', 'radius_err_c', 
                                               'mass_c', 'mass_err_c', 'mv', 'bvcolor', 'parallax']]

In [13]:
stellar_rounded.to_csv('legacy_tables/stellar_parameters_rounded.csv')

In [14]:
masses_only.columns

Index(['Unnamed: 0', 'hostname', 'status', 'pl_index', 'mass', 'mass_med',
       'mass_minus', 'mass_plus', 'axis', 'axis_med', 'axis_minus',
       'axis_plus', 'per', 'per_med', 'per_minus', 'per_plus', 'tc', 'tc_med',
       'tc_minus', 'tc_plus', 'w', 'w_med', 'w_minus', 'w_plus', 'k', 'k_med',
       'k_minus', 'k_plus', 'e', 'e_med', 'e_minus', 'e_plus', 'e_mode',
       'e_68', 'insol', 'insol_med', 'insols_minus', 'insols_plus', 'teq',
       'teq_med', 'teq_minus', 'teq_plus', 'post_path'],
      dtype='object')

In [36]:
masses_only = masses_only[['hostname', 'status', 'pl_index', 'mass', 'mass_med',
       'mass_minus', 'mass_plus', 'axis', 'axis_med', 'axis_minus',
       'axis_plus', 'per', 'per_med', 'per_minus', 'per_plus', 'tc', 'tc_med',
       'tc_minus', 'tc_plus', 'w', 'w_med', 'w_minus', 'w_plus', 'k', 'k_med',
       'k_minus', 'k_plus', 'e', 'e_med', 'e_minus', 'e_plus', 'e_mode', 'e_68']]

In [37]:
planets_rounded = masses_only.round(8)

In [38]:
planets_rounded = planets_rounded.sort_values('hostname').reset_index(drop=True)

In [39]:
planets_rounded.to_csv('legacy_tables/planet_list_rounded.csv')

In [47]:
data_select_rounded = data_select.round(6)

In [48]:
data_select_rounded

,name,tel,obnm,jd,mnvel,errvel,sval,cts
0,10002,k,rk9.173,2.450463e+06,-2.726316,0.977371,NaN,94095.0
1,10002,k,rk15.119,2.450715e+06,-7.562192,1.080576,NaN,128078.0
2,10002,k,rk17.27,2.450806e+06,-6.910398,1.073604,NaN,105461.0
3,10002,k,rk17.172,2.450807e+06,-2.655050,1.173251,NaN,106530.0
4,10002,k,rk18.25,2.450838e+06,-6.846250,1.069229,NaN,168101.0
...,...,...,...,...,...,...,...,...
104715,223691,j,rj02.159,2.453302e+06,-8.438038,1.038896,0.1450,97462.0
104716,223691,j,rj11.996,2.453605e+06,5.800885,0.957615,0.1450,98349.0
104717,223691,j,rj231.531,2.457571e+06,2.241184,1.557648,0.1487,16903.0
104718,223691,j,rj278.1062,2.458118e+06,-2.181665,1.199874,0.1454,22289.0


In [49]:
data_select_rounded.to_csv('legacy_tables/legacy_data_rounded.csv')

In [50]:
apfphot = pd.read_csv('legacy_tables/apf50_photometry.csv')

In [52]:
apfphot_rounded = apfphot.round(6)

In [53]:
apfphot_rounded.to_csv('legacy_tables/apf60_photometry_rounded.csv')

In [4]:
trends

,Unnamed: 0,index,hostname,mstar,dvdt,dvdt_med,dvdt_minus,dvdt_plus,curv,curv_med,curv_minus,curv_plus
0,0,2,100623,0.747831,0.004761,0.004752,0.004473,0.005039,NaN,NaN,NaN,NaN
1,1,27,110315,0.718441,-0.055436,-0.055450,-0.055844,-0.055067,0.000002,0.000002,0.000002,0.000002
2,2,28,110537,0.995681,0.019294,0.019365,0.018867,0.019889,NaN,NaN,NaN,NaN
3,3,35,114174,0.968486,0.162774,0.162784,0.162219,0.163379,-0.000002,-0.000003,-0.000003,-0.000002
4,4,41,115404a,0.780834,0.015565,0.015573,0.014551,0.016633,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
84,84,647,151995,0.774024,-0.006613,-0.006426,-0.007244,-0.005525,NaN,NaN,NaN,NaN
85,85,651,156826,1.372675,-0.024066,-0.024204,-0.025148,-0.023406,NaN,NaN,NaN,NaN
86,86,677,180684,1.262973,0.171002,0.170927,0.169106,0.172823,-0.000007,-0.000007,-0.000007,-0.000006
87,87,691,201203,1.459537,-0.144294,-0.144700,-0.154866,-0.135465,-0.000032,-0.000031,-0.000036,-0.000027


In [5]:
trends_rounded = trends.round(10)

In [10]:
trends_rounded.to_csv('legacy_tables/trends_list_rounded.csv')